In [1]:
import warnings
warnings.filterwarnings("ignore")

import logging, sys
logging.disable(sys.maxsize)

import numpy as np

from pyaxis import pyaxis

import matplotlib.pyplot as plt
from matplotlib import style
plt.style.use('seaborn-darkgrid')

from sklearn.linear_model import LinearRegression

In [2]:
# Naravni prirast
url = './naravni_prirast.PX'
px = pyaxis.parse(url, encoding='UTF-8')
df = px['DATA']

# Naravni prirast Slovenija 1922 - 2020
url = './pop_slovenija.PX'
px4 = pyaxis.parse(url, encoding='Windows-1250')
df4 = px4["DATA"]

# Izobrazba
url = './izobrazba.PX'
px01 = pyaxis.parse(url, encoding='Windows-1250')
df1 = px01["DATA"]

# Place
url = './povprecne_place.PX'
px2 = pyaxis.parse(url, encoding='ISO-8859-2')
df2 = px2["DATA"]

# Selitve
url = './selitve.PX'
px3 = pyaxis.parse(url, encoding='Windows-1250')
df3 = px3["DATA"]

# Rodnost
url = './tfr.PX'
px5 = pyaxis.parse(url, encoding='Windows-1250')
df5 = px5["DATA"]

# Veroizpoved
url = './veroizpoved2002.PX'
px6 = pyaxis.parse(url, encoding='Windows-1250')
df6 = px6["DATA"]

# Selitve
url = './selitve.PX'
px7 = pyaxis.parse(url, encoding='Windows-1250')
df7 = px7["DATA"]

In [3]:
def naravni_prirast_slo_2220_1000():
    slo_np22 = {}
    for i in range(len(df4)):
        if df4.loc[i, "MERITVE"] == "Naravni prirast na 1.000 prebivalcev":
            slo_np22[df4.loc[i, "LETO"]] = float(df4.loc[i, "DATA"])
    
    plt.figure(figsize=(20, 5))
    plt.title("Naravni prirast Slovenije na 1000 prebivalcev (1922-2020)", fontsize=14, fontweight='bold')

    plt.xlim(-0.5, len(slo_np22.keys()))
    plt.xticks(range(len(slo_np22.keys())))
    plt.gca().set_xticklabels(slo_np22.keys(), rotation=90)
    plt.xlabel('Leto'); 
    plt.ylabel('Naravni prirast na 1000 prebivalcev');
    plt.plot(slo_np22.keys(), slo_np22.values(), 'g--', color='b');
    print("Naravni prirast leta 1941:", slo_np22["1941"], "\nNaravni prirast leta 1945:", slo_np22["1945"])

In [4]:
def naravni_prirast_obcine_9720_1000():
    np_obcine = {}
    leta_kraji = {}

    for i in range(len(df)):
        if df.loc[i, "MERITVE"] == "Naravni prirast na 1.000 prebivalcev" and df.loc[i, "OBCINE"] != "SLOVENIJA":
            if not df.loc[i, "DATA"] == '"-"' and not df.loc[i, "DATA"] == float("nan") and int(df.loc[i, "LETO"]) > 1997:
                if df.loc[i, "OBCINE"] not in np_obcine.keys():
                    np_obcine[df.loc[i, "OBCINE"]] = []
                    leta_kraji[df.loc[i, "OBCINE"]] = []
                np_obcine[df.loc[i, "OBCINE"]].append(float(df.loc[i, "DATA"]))
                leta_kraji[df.loc[i, "OBCINE"]].append([df.loc[i, "LETO"], df.loc[i, "DATA"]])
    
    np_obcine_avg = {}
    for obcina in np_obcine:
        np_obcine_avg[obcina] = np.nanmean(np_obcine[obcina])
        
    print ("Prirastek občine z največjim povprečnim naravnim prirastkom:", max(np_obcine_avg, key=np_obcine_avg.get), ",", np_obcine_avg[max(np_obcine_avg, key=np_obcine_avg.get)])
    print ("Prirastek občine z najmanjšim povprečnim naravnim prirastkom:", min(np_obcine_avg, key=np_obcine_avg.get), ",", np_obcine_avg[min(np_obcine_avg, key=np_obcine_avg.get)])
    print ("Število občin z pozitivnim nar. prirastkom:", len([x for x in np_obcine_avg.values() if x > 0]))
    
    komenda = np_obcine["Komenda"]
    komenda = [d for d in komenda if str(d) != "nan"]
    hodos = np_obcine["Hodos"]
    hodos = [d for d in hodos if str(d) != "nan"]

    np_minmax = {}
    i = 1999
    for x, y in zip(komenda, hodos):
        np_minmax[i] = [x, y]
        i += 1

    komenda = np.array(list(np_minmax.values()))[:,0]
    hodos = np.array(list(np_minmax.values()))[:,1]

    plt.figure(figsize=(8, 4))

    plt.xlabel('Year'); 
    plt.ylabel('Population'); 
    plt.title("Naravno gibanje občin Komenda in Hodoš")
    plt.plot(np_minmax.keys(), komenda, color='b', label="Komenda");
    plt.plot(np_minmax.keys(), hodos, color='r', label="Hodoš");
    plt.legend()
    plt.show()

In [5]:
def naravni_prirast_slo_9720_mz():
    np_moski = []
    np_zenske = []

    for i in range(len(df)):
        if df.loc[i, "OBCINE"] == "SLOVENIJA" and df.loc[i, "MERITVE"] == "Naravni prirast - moski":
            np_moski.append(int(df.loc[i, "DATA"]))
        if df.loc[i, "OBCINE"] == "SLOVENIJA" and df.loc[i, "MERITVE"] == "Naravni prirast - zenske":
            np_zenske.append(int(df.loc[i, "DATA"]))

    #print("Naravni prirast moski: ", np.mean(np_moski))
    #print("Naravni prirast zenske: ", np.mean(np_zenske))

    np_mzdict = {}
    i = 1995
    for x, y in zip(np_moski, np_zenske):
        np_mzdict[i] = [x, y]
        i += 1

    np_moski = np.array(list(np_mzdict.values()))[:,0]
    np_zenske = np.array(list(np_mzdict.values()))[:,1]
    # print(np_mzdict)

    plt.figure(figsize=(8, 4))

    plt.xlabel('Leto'); 
    plt.ylabel('Naravni prirastek'); 
    plt.title("Naravno gibanje prebivalstva Slovenije (moški in ženske ločeno)")
    plt.plot(np_mzdict.keys(), np_moski, color='b', label="Moski");
    plt.plot(np_mzdict.keys(), np_zenske, color='r', label="Zenske");
    plt.legend()
    plt.show()

In [6]:
def izobrazba_2020():
    osns, sred, vis = 0, 0, 0
    for i in range(len(df1)):
        if df1.loc[i, "OBCINE"] == "SLOVENIJA" and df1.loc[i, "SPOL"] == "Spol - SKUPAJ" and int(df1.loc[i, "LETO"]) == 2020:
            if df1.loc[i, "IZOBRAZBA"] == "Osnovnosolska ali manj - Skupaj":
                osns = int(df1.loc[i, "DATA"])
            elif df1.loc[i, "IZOBRAZBA"] == "Srednjesolska - Skupaj":
                sred = int(df1.loc[i, "DATA"])
            elif df1.loc[i, "IZOBRAZBA"] == "Visjesolska, visokosolska - Skupaj":
                vis = int(df1.loc[i, "DATA"])
    print("Leta 2020:")      
    print("- Osnovnosolska ali manj:", osns, "\n- Srednjesolska:", sred, "\n- Visjesolska, visokosolska:", vis)
    print()
    
    osns, sred, vis = [], [], []
    for i in range(len(df1)):
        if df1.loc[i, "OBCINE"] == "SLOVENIJA" and df1.loc[i, "SPOL"] == "Spol - SKUPAJ":
            if df1.loc[i, "IZOBRAZBA"] == "Osnovnosolska ali manj - Skupaj":
                osns.append(int(df1.loc[i, "DATA"]))
            elif df1.loc[i, "IZOBRAZBA"] == "Srednjesolska - Skupaj":
                sred.append(int(df1.loc[i, "DATA"]))
            elif df1.loc[i, "IZOBRAZBA"] == "Visjesolska, visokosolska - Skupaj":
                vis.append(int(df1.loc[i, "DATA"]))

    iz = {}
    iz["osns"] = round(np.mean(osns))
    iz["sred"] = round(np.mean(sred))
    iz["vis"] = round(np.mean(vis))
    
    print("Povprecje 2011-2021:")      
    print("- Osnovnosolska ali manj:", iz["osns"], "\n- Srednjesolska:", iz["sred"], "\n- Visjesolska, visokosolska:", iz["vis"])

In [7]:
def bruto_placa_letno_slo():
    np2_pla = {}
    for i in range(len(df2)):
        if df2.loc[i, "OBCINE"] == "SLOVENIJA" and df2.loc[i, "KAZALNIK"] == "Povprecna mesecna bruto placa":
            np2_pla[str(df2.loc[i, "LETO"])] = float(df2.loc[i, "DATA"])
            
    plt.figure(figsize=(8, 4))
    plt.title('Povprečna bruto plača (Slovenija 2005-2021)', fontsize=14, fontweight='bold')
    plt.bar(np2_pla.keys(), np2_pla.values(), color='g')
    plt.xlim(-0.5, len(np2_pla.keys())-0.5)
    plt.xticks(range(len(np2_pla.keys())))
    plt.gca().set_xticklabels(np2_pla.keys(), rotation=90)
    plt.xlabel("Leto")
    plt.ylabel("Plača")
    plt.show()

In [8]:
def inflacija(stopnja, dva5):
    inf_r = stopnja / 100
    print("Vrednost povprečne plače leta 2005:", dva5, "\nVrednost plače iz leta 2005 leta 2021 z upoštevanjem inflacije:", round(inf_r * dva5 + dva5,2))

In [9]:
def min_max_bruto_placa_obcine():
    # 2020
    np2_obcine = {}
    for i in range(len(df2)):
        if df2.loc[i, "KAZALNIK"] == "Povprecna mesecna bruto placa" and int(df2.loc[i, "LETO"]) == 2020:
            np2_obcine[df2.loc[i, "OBCINE"]] = float(df2.loc[i, "DATA"])
            
    print("Leta 2020:")
    print(" -", max(np2_obcine, key=np2_obcine.get), ":", float(np.nanmean(np2_obcine[max(np2_obcine, key=np2_obcine.get)])))
    print(" -", min(np2_obcine, key=np2_obcine.get), ":", float(np.nanmean(np2_obcine[min(np2_obcine, key=np2_obcine.get)])))
    print()
    
    # Povprecje
    np2_obcine = {}
    for i in range(len(df2)):
        if df2.loc[i, "KAZALNIK"] == "Povprecna mesecna bruto placa":
            if df2.loc[i, "OBCINE"] not in np2_obcine.keys():
                np2_obcine[df2.loc[i, "OBCINE"]] = []
            try:
                np2_obcine[df2.loc[i, "OBCINE"]].append(float(df2.loc[i, "DATA"]))
            except ValueError:
                np2_obcine[df2.loc[i, "OBCINE"]].append(float("nan"))
    
    print("Povprecje 2005-2021:")
    print(" -", max(np2_obcine, key=np2_obcine.get), ":", float(np.nanmean(np2_obcine[max(np2_obcine, key=np2_obcine.get)])))
    print(" -", min(np2_obcine, key=np2_obcine.get), ":", float(np.nanmean(np2_obcine[min(np2_obcine, key=np2_obcine.get)])))

In [10]:
# 1.vprasanje
def napoved_lin_reg():
    slo_np22 = {}
    for i in range(len(df4)):
        if df4.loc[i, "MERITVE"] == "Naravni prirast na 1.000 prebivalcev":
            slo_np22[df4.loc[i, "LETO"]] = float(df4.loc[i, "DATA"])
            
    x = np.array(list(slo_np22.keys())).reshape(-1,1)
    y = np.array(list(slo_np22.values())).reshape(-1,1)

    # Linearna regresija
    model = LinearRegression()
    model.fit(x, y)
    pass

    plt.figure(figsize=(20, 5))
    plt.title("Naravni prirast Slovenije na 1000 prebivalcev (1922-2020)", fontsize=14, fontweight='bold')

    plt.xlim(-0.5, len(slo_np22.keys()))
    plt.xticks(range(len(slo_np22.keys())))
    plt.gca().set_xticklabels(slo_np22.keys(), rotation=90)
    plt.xlabel('Leto'); 
    plt.ylabel('Naravni prirast na 1000 prebivalcev');
    plt.plot(slo_np22.keys(), slo_np22.values(), 'rx', color='b');

    # Premica
    hx = model.predict(x)
    plt.plot(x.ravel(), hx.ravel(), "r-", label="Model")
    plt.show()
    
    hx = model.predict([[2050]])
    print("Napoved za leto 2050:", round(float(hx[0][0]),2))

In [11]:
# 2. vprasanje
def povp_placa_zivorojeni():
    # 5 let
    place = {}
    for i in range(len(df2)):
        if df2.loc[i, "KAZALNIK"] == "Povprecna mesecna bruto placa" and df2.loc[i, "OBCINE"] != "SLOVENIJA" and 2020 > int(df2.loc[i, "LETO"]) > 2014:
            if df2.loc[i, "OBCINE"] not in place.keys():
                place[df2.loc[i, "OBCINE"]] = []
            place[df2.loc[i, "OBCINE"]].append(float(df2.loc[i, "DATA"]))

    # povprecje bruto plac po obcinah
    place_avg = {}
    for obcina in place:
        place_avg[obcina] = round(np.mean(place[obcina]), 2)
    
    
    # 5 let
    zivo_r = {}
    for i in range(len(df)):
        if df.loc[i, "MERITVE"] == "Zivorojeni na 1.000 prebivalcev" and df.loc[i, "OBCINE"] != "SLOVENIJA" and 2020 > int(df.loc[i, "LETO"]) > 2014:
            if df.loc[i, "OBCINE"] not in zivo_r.keys():
                zivo_r[df.loc[i, "OBCINE"]] = []
            zivo_r[df.loc[i, "OBCINE"]].append(float(df.loc[i, "DATA"]))

    # povprecje zivorojenih po obcinah
    zivo_r_avg = {}
    for ziv in zivo_r:
        zivo_r_avg[ziv] = round(np.mean(zivo_r[ziv]), 2)
        
    # [placa, zivorojeni]
    pla_ziv = {}
    for obcina in place_avg:
        pla_ziv[obcina] = [place_avg[obcina], zivo_r_avg[obcina]]

    # V np.array
    tmp = np.array(list(pla_ziv.values()))
    tmp = tmp[tmp[:,0] < 2330.7] # osamelec
    x = tmp[:,0]
    y = tmp[:,1]
    
        # Korelacija - Pearson
    r = np.corrcoef(x,y)

    plt.figure(figsize=(12, 5))
    plt.xlabel('Povprečna bruto plača')
    plt.ylabel('Število živorojenih na 1000 prebivalcev')
    plt.plot(x,y,'o')
    m, b = np.polyfit(x, y, 1)
    plt.plot(x, m*x + b)
    plt.show()
    
    print("Koeficient korelacije:\n", round(float(r[0][1]), 4))

In [12]:
# 2. vprasanje
def povp_placa_rodnost():
    
    # rodnost 2018
    ftr_obcine = {}
    for i in range(len(df5)):
        if df5.loc[i, "OBCINE"] != "SLOVENIJA" and df5.loc[i, "LETO"] == "2018" and df5.loc[i, "KAZALNIKI"] == "Povprecno stevilo zivorojenih otrok na zensko":
            ftr_obcine[df5.loc[i, "OBCINE"]] = float(df5.loc[i, "DATA"])
            
    # place 2018
    place = {}
    for i in range(len(df2)):
        if df2.loc[i, "KAZALNIK"] == "Povprecna mesecna bruto placa" and df2.loc[i, "OBCINE"] != "SLOVENIJA" and int(df2.loc[i, "LETO"]) == 2018:
            place[df2.loc[i, "OBCINE"]] = float(df2.loc[i, "DATA"])
            
    # [placa, ftr]
    pla_ftr = {}
    for obcina in place:
        pla_ftr[obcina] = [place[obcina], ftr_obcine[obcina]]

    # V np.array
    tmp = np.array(list(pla_ftr.values()))

    # z-score -> osamelci
    from scipy import stats
    z = np.abs(stats.zscore(tmp[:,0]))
    o = np.where(z > 3)
    tmp = np.delete(tmp, o[0][0], axis=0)

    x = tmp[:,0]
    y = tmp[:,1]
    
    # Korelacija - Pearson
    r = np.corrcoef(x,y)
    
    plt.figure(figsize=(12, 5))
    plt.xlabel('Povprečna bruto plača')
    plt.ylabel('Rodnost')
    plt.plot(x,y,'o')
    m, b = np.polyfit(x, y, 1)
    plt.plot(x, m*x + b)
    plt.show()
    
    print("Koeficient korelacije:\n", round(float(r[0][1]), 4))

In [13]:
# 3. vprasanje
def izobrazba_nar_prirast():
    import logging, sys
    logging.disable(sys.maxsize)

    import pandas as pd
    from pyaxis import pyaxis

    import matplotlib.pyplot as plt
    from matplotlib import style
    plt.style.use('seaborn-darkgrid')
    url2 = './izobrazba.PX'
    izo = pyaxis.parse(url2, encoding='Windows-1250')
    izobrazba = izo["DATA"]
    izobrazba = np.array(izobrazba)
    url = './naravni_prirast.PX'
    px = pyaxis.parse(url, encoding='UTF-8')
    df = px['DATA']
    naravni = np.array(df)

    #iskanje obcin
    listObcin = set()
    for _,obcina,_,_,_ in izobrazba:
        if obcina != "SLOVENIJA":
            listObcin.add(obcina)    
    #iskanje  izobrazbe
    slovar = {}
    glavneStopnje = ["Osnovnosolska ali manj - Skupaj", "Srednjesolska - Skupaj", "Visjesolska, visokosolska - Skupaj"]
    for obcina in listObcin:
        zacas = []
        for stopnja in glavneStopnje:
            #tukej poiscemo use podatke za zenske z usako obcino od leta 2016 do 2020, kjer je stopnja "Osnovnosolska ali manj - Skupaj", "Srednjesolska - Skupaj" ali "Visjesolska, visokosolska - Skupaj"
            #vrednosti nato pretvorimo v float
            temp = [((izobrazba[(izobrazba[:,0] == "Zenske") * (izobrazba[:,1] == obcina) * (izobrazba[:,2] >= "2016") * (izobrazba[:,2] < "2021") * (izobrazba[:,3] == stopnja),4 ]))]
            temp = [float(x) for x in temp[0]]
            zacas.append(temp)
        slovar[obcina] = zacas



    slovarPrirast = {}
    #iskanje naravnega prirasta
    for obcina in listObcin:
        zacas = []
        #tukej poiscemo use podatke za usako obcino od leta 2016 do 2020, za naravni prirast na 1000 prebivalcev ter nato vrednosti pretvorimo v float
        zacas = (naravni[(naravni[:,0] == obcina) * (naravni[:,1] >= "2016") * (naravni[:,1] < "2021") * (naravni[:,2]) == "Naravni prirast - Skupaj",3]) #"Naravni prirast na 1.000 prebivalcev
        zacas = [float(x) for x in zacas]
        slovarPrirast[obcina] = [zacas]

    corOsnovna = {}
    corSrednja = {}
    corVisoka = {}
    #IZRAČUN KORELACIJE IN POVPREČENJE
    for obcina in listObcin:
        corOsnovna[obcina] = [np.mean(slovar[obcina][0]),np.mean(slovarPrirast[obcina][0])]
        corSrednja[obcina] = [np.mean(slovar[obcina][1]),np.mean(slovarPrirast[obcina][0])]
        corVisoka[obcina] = [np.mean(slovar[obcina][2]),np.mean(slovarPrirast[obcina][0])]
    tmpOsn = np.array(list(corOsnovna.values()))
    xOsn = tmpOsn[:,0]
    yOsn = tmpOsn[:,1]
    print(np.corrcoef(xOsn,yOsn)[0][1])

    # Graf
    plt.figure(figsize=(12, 5))
    plt.xlabel('Št. ljudi z osnovnošolsko izobrazbo')
    plt.ylabel('Naravni prirast')
    plt.plot(xOsn,yOsn,'o')
    m, b = np.polyfit(xOsn, yOsn, 1)
    plt.plot(xOsn, m*xOsn + b)
    plt.show()

    tmpSred = np.array(list(corSrednja.values()))
    xSred = tmpSred[:,0]
    ySred = tmpSred[:,1]
    print(np.corrcoef(xSred,ySred)[0][1])
    
    # Graf
    plt.figure(figsize=(12, 5))
    plt.xlabel('Št. ljudi s srednješolsko izobrazbo')
    plt.ylabel('Naravni prirast')
    plt.plot(xSred,ySred,'o')
    m, b = np.polyfit(xSred, ySred, 1)
    plt.plot(xSred, m*xSred + b)
    plt.show()

    tmpVs = np.array(list(corVisoka.values()))
    xVs = tmpVs[:,0]
    yVs = tmpVs[:,1]
    print(np.corrcoef(xVs,yVs)[0][1])
    
    # Graf
    plt.figure(figsize=(12, 5))
    plt.xlabel('Št. ljudi z visoko/višješolsko izobrazbo')
    plt.ylabel('Naravni prirast')
    plt.plot(xVs,yVs,'o')
    m, b = np.polyfit(xVs, yVs, 1)
    plt.plot(xVs, m*xVs + b)
    plt.show()

In [14]:
def rojeni_vera_obcine():
    # vera
    obcine2002 = {}
    for i in range(len(df6)):
        if df6.loc[i, "OBCINA"] != "SLOVENIJA":
            if df6.loc[i, "OBCINA"] not in obcine2002.keys():
                obcine2002[df6.loc[i, "OBCINA"]] = [0, []]
            if df6.loc[i, "VEROIZPOVED"] == "Veroizpoved - SKUPAJ":
                obcine2002[df6.loc[i, "OBCINA"]][0] = df6.loc[i, "DATA"]

            if df6.loc[i, "VEROIZPOVED"] == "katoliska":
                obcine2002[df6.loc[i, "OBCINA"]][1].append(float(df6.loc[i, "DATA"]))
            if df6.loc[i, "VEROIZPOVED"] == "evangelicanska in druge protestantske":
                obcine2002[df6.loc[i, "OBCINA"]][1].append(float(df6.loc[i, "DATA"]))
            if df6.loc[i, "VEROIZPOVED"] == "pravoslavna":
                obcine2002[df6.loc[i, "OBCINA"]][1].append(float(df6.loc[i, "DATA"]))
            if df6.loc[i, "VEROIZPOVED"] == "islamska":
                obcine2002[df6.loc[i, "OBCINA"]][1].append(float(df6.loc[i, "DATA"]))
            if df6.loc[i, "VEROIZPOVED"] == "druge veroizpovedi":
                obcine2002[df6.loc[i, "OBCINA"]][1].append(float(df6.loc[i, "DATA"]))
            if df6.loc[i, "VEROIZPOVED"] == "Je vernik, ne pripada nobeni veroizpovedi":
                obcine2002[df6.loc[i, "OBCINA"]][1].append(float(df6.loc[i, "DATA"]))

    #print(obcine2002)
    never_delez = {}
    for obcina in obcine2002:
        never_delez[obcina] = 1 - int(np.nansum(obcine2002[obcina][1])) / int(obcine2002[obcina][0])
    
    
    # Fertility
    ftr_obcine2002 = {}
    for i in range(len(df5)):
        if df5.loc[i, "OBCINE"] != "SLOVENIJA" and df5.loc[i, "LETO"] == "2011" and df5.loc[i, "KAZALNIKI"] == "Povprecno stevilo zivorojenih otrok na zensko":
            ftr_obcine2002[df5.loc[i, "OBCINE"]] = float(df5.loc[i, "DATA"])

    ziv_ver = {}
    for obcina in never_delez:
        ziv_ver[obcina] = [never_delez[obcina], ftr_obcine2002[obcina]]

    # V np.array
    tmp = np.array(list(ziv_ver.values()))

    x = tmp[:,0]
    y = tmp[:,1]
    
    # Korelacija - Pearson
    r = np.corrcoef(x,y)
    
    print(np.corrcoef(x,y)[0][1])
    
    # Graf
    plt.figure(figsize=(12, 5))
    plt.xlabel('Delež nevernih državljanov')
    plt.ylabel('Rodnost')
    plt.plot(x,y,'o')
    m, b = np.polyfit(x, y, 1)
    plt.plot(x, m*x + b)
    plt.show()

In [15]:
# selitve in izselitve
def slo_tujina():
    v_slo = []
    iz_slo = []

    # selitve v slovenijo
    for i in range(len(df7)):
        # zadnjih 5 let priselitve
        if df7.loc[i, "MERITVE"] == "Priseljeni iz tujine - Skupaj" and df7.loc[i, "OBCINE"] == "SLOVENIJA":
            v_slo.append(int(float(df7.loc[i, "DATA"])))

    # selitve iz slovenije
    for i in range(len(df7)):
        # zadnjih 5 let odselitve
        if df7.loc[i, "MERITVE"] == "Odseljeni v tujino - Skupaj" and df7.loc[i, "OBCINE"] == "SLOVENIJA":
            iz_slo.append(-int(float(df7.loc[i, "DATA"])))
            
    c = [a for a in range(1995,2021)]
    x = range(len(c))

    plt.figure(figsize=(12, 5))
    plt.bar(x, iz_slo, color='r', label='odselitve')
    plt.bar(x, v_slo, color='b', label='priselitve')
    plt.xlim(-0.5, len(x)-0.5)
    plt.xticks(x)
    plt.gca().set_xticklabels(c, rotation=90)
    plt.legend()
    plt.show()

In [16]:
def tujina_obcine():
    pris_tujina1000 = {}
    for i in range(len(df7)):
        # zadnjih 5 let priselitve
        if df7.loc[i, "MERITVE"] == "Priseljeni iz tujine na 1000 prebivalcev" and df7.loc[i, "OBCINE"] != "SLOVENIJA" and int(df7.loc[i, "LETO"]) > 2015:
            if df7.loc[i, "OBCINE"] not in pris_tujina1000.keys():
                pris_tujina1000[df7.loc[i, "OBCINE"]] = []
            pris_tujina1000[df7.loc[i, "OBCINE"]].append(float(df7.loc[i, "DATA"]))
    
    ods_tujina1000 = {}
    for i in range(len(df7)):
        # zadnjih 5 let priselitve
        if df7.loc[i, "MERITVE"] == "Odseljeni v tujino na 1000 prebivalcev" and df7.loc[i, "OBCINE"] != "SLOVENIJA" and int(df7.loc[i, "LETO"]) > 2015:
            if df7.loc[i, "OBCINE"] not in ods_tujina1000.keys():
                ods_tujina1000[df7.loc[i, "OBCINE"]] = []
            ods_tujina1000[df7.loc[i, "OBCINE"]].append(float(df7.loc[i, "DATA"]))
            
    for obcina in pris_tujina1000:
        pris_tujina1000[obcina] = np.mean(pris_tujina1000[obcina])
    
    for obcina in ods_tujina1000:
        ods_tujina1000[obcina] = np.mean(ods_tujina1000[obcina])
        
    # max in min - popravi izpis
    print("Priselitve, max in min")
    print("----------------------")
    a = max(pris_tujina1000, key=pris_tujina1000.get)
    b = pris_tujina1000[max(pris_tujina1000, key=pris_tujina1000.get)]
    print(a, b)
    a = min(pris_tujina1000, key=pris_tujina1000.get)
    b = pris_tujina1000[min(pris_tujina1000, key=pris_tujina1000.get)]
    print(a, b)
    print()
    print("Odselitve, max in min")
    print("----------------------")
    a = max(ods_tujina1000, key=ods_tujina1000.get)
    b = ods_tujina1000[max(ods_tujina1000, key=ods_tujina1000.get)]
    print(a, b)
    a = min(ods_tujina1000, key=ods_tujina1000.get)
    b = ods_tujina1000[min(ods_tujina1000, key=ods_tujina1000.get)]
    print(a, b)

In [17]:
def obcine_obcine():
    pris_obcine1000 = {}
    for i in range(len(df7)):
        # zadnjih 5 let priselitve
        if df7.loc[i, "MERITVE"] == "Priseljeni iz drugih obcin na 1000 prebivalcev" and df7.loc[i, "OBCINE"] != "SLOVENIJA" and int(df7.loc[i, "LETO"]) > 2015:
            if df7.loc[i, "OBCINE"] not in pris_obcine1000.keys():
                pris_obcine1000[df7.loc[i, "OBCINE"]] = []
            pris_obcine1000[df7.loc[i, "OBCINE"]].append(float(df7.loc[i, "DATA"]))
            
    ods_obcine1000 = {}
    for i in range(len(df7)):
        # zadnjih 5 let priselitve
        if df7.loc[i, "MERITVE"] == "Odseljeni v druge obcine na 1000 prebivalcev" and df7.loc[i, "OBCINE"] != "SLOVENIJA" and int(df7.loc[i, "LETO"]) > 2015:
            if df7.loc[i, "OBCINE"] not in ods_obcine1000.keys():
                ods_obcine1000[df7.loc[i, "OBCINE"]] = []
            ods_obcine1000[df7.loc[i, "OBCINE"]].append(float(df7.loc[i, "DATA"]))
            
    for obcina in pris_obcine1000:
        pris_obcine1000[obcina] = np.mean(pris_obcine1000[obcina])
    
    for obcina in ods_obcine1000:
        ods_obcine1000[obcina] = np.mean(ods_obcine1000[obcina])
        
    # max in min - popravi izpis
    print("Priselitve, max in min")
    print("----------------------")
    a = max(pris_obcine1000, key=pris_obcine1000.get)
    b = pris_obcine1000[max(pris_obcine1000, key=pris_obcine1000.get)]
    print(a, b)
    a = min(pris_obcine1000, key=pris_obcine1000.get)
    b = pris_obcine1000[min(pris_obcine1000, key=pris_obcine1000.get)]
    print(a, b)
    print()
    print("Odselitve, max in min")
    print("----------------------")
    a = max(ods_obcine1000, key=ods_obcine1000.get)
    b = ods_obcine1000[max(ods_obcine1000, key=ods_obcine1000.get)]
    print(a, b)
    a = min(ods_obcine1000, key=ods_obcine1000.get)
    b = ods_obcine1000[min(ods_obcine1000, key=ods_obcine1000.get)]
    print(a, b)

In [18]:
def sel_nar_pr():
    sk_sel1000 = {}
    for i in range(len(df7)):
        if df7.loc[i, "MERITVE"] == "Skupni selitveni prirast na 1000 prebivalcev" and df7.loc[i, "OBCINE"] != "SLOVENIJA" and 2020 == int(df7.loc[i, "LETO"]):
            sk_sel1000[df7.loc[i, "OBCINE"]] = float(df7.loc[i, "DATA"])
            
    np_1000_5 = {}
    for i in range(len(df)):
        if df.loc[i, "MERITVE"] == "Naravni prirast na 1.000 prebivalcev" and df.loc[i, "OBCINE"] != "SLOVENIJA" and 2020 == int(df.loc[i, "LETO"]):
            np_1000_5[df.loc[i, "OBCINE"]] = float(df.loc[i, "DATA"])
            
    np_sel = {}
    for obcina in sk_sel1000:
        np_sel[obcina] = [sk_sel1000[obcina], np_1000_5[obcina]]

    # V np.array
    tmp = np.array(list(np_sel.values()))
    tmp = tmp[tmp[:,0] < 100]
    tmp = tmp[tmp[:,0] > -40]

    x = tmp[:,0]
    y = tmp[:,1]
    
    r = np.corrcoef(x,y)
    print("Koeficient korelacije:", r[0][1])
    
    plt.figure(figsize=(12, 5))
    plt.xlabel('Selitve/1000')
    plt.ylabel('Naravni prirast/1000')
    plt.plot(x,y,'o')
    m, b = np.polyfit(x, y, 1)
    plt.plot(x, m*x + b)
    plt.show()